## Social Media Monitoring and Analysis

Welcome! This notebook is geared to give you starter examples of how to use ChatGPT for social media monitoring and analysis. Specifically we will look at three use cases:
- Sentiment and Emotion Analysis
- Topic Extraction
- Classification / Categorization

For our purposes we will assume we work at Sam's Club (a well-known retail warehouse chain owned by Wal-Mart) and have been asked to look at a series of tweets for our analysis. I'll walk you through all the basic steps needed to do the analysis. 

NOTE: To make it easier to focus on the process of analysis instead of acquiring the data, I've included a JSON file that has curated output from Twitter (now called, X) in it. If you want to sign up for a developer account so you can get live data, go here: https://developer.twitter.com/

### Sentiment and Emotion Analysis

The first order of business is to take the data and put it into a format that is easier to analyze. Take a moment to look at the Tweets.json file and see how we get information from Twitter. Notice that the file is a list of dictionaries where each dictionary represents a tweet. We need to manipulate it into something usable. Let's begin!

In [3]:
# import our packages 
import pandas as pd
import json
import os
import openai

# set our key for openai to be used later
openai_key = os.getenv('OPENAI_KEY')
openai.api_key = openai_key


### Read the Data

Now that we have our packages ready to go and our OpenAI API key set we need to read the data info a pandas dataframe to ease our cleaning and analysis efforts.

In [5]:
# Start by inspecting the json file and determining the structure of the data. Remember that the file is a list of dictionaries where each dictionary represents a tweet. 
# Some are nested dictionaries and some are not. We need to flatten the data so that we can convert it into a DataFrame.
# The json_normalize function from pandas is used to convert JSON data into a flat table (DataFrame).
from pandas import json_normalize

# Open the file named "Tweets.json" for reading. The "with" statement ensures that the file is properly closed after it is no longer needed.
with open("Tweets.json") as file:
    # Use the json.load function to load the JSON data from the file into a Python object (usually a list or a dictionary).
    data = json.load(file)

# Convert the JSON data into a DataFrame. json_normalize flattens the data, meaning it can create a DataFrame from nested JSON data.
df = json_normalize(data)

# Print the first 5 rows of the DataFrame to see what the data looks like.
print(df.head())

# Save the DataFrame to a CSV file named "initaldataframe.csv" so you can see the progress. The argument index=False means that the DataFrame's index will not be saved in the CSV file.
# While this step isn't necessary, it's a good idea to save your work as you go along and to check that the data looks correct.
# If you are using VS Code, I highly recommend installing the Excel Viewer extension or Rainbow CSV extension so you can view the CSV file more easily.
df.to_csv('initaldataframe.csv', index=False)


                       created_at                   id               id_str  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793  1658153870575521793   

                                           full_text  truncated  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...      False   
1           @KellieShai_ All we need is some milk. 🍪      False   
2  Happy National Chocolate Chip Day! No denying ...      False   
3  This is heartbreaking that less than a year im...      False   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...      False   

  display_text_range                                             source  \
0           [0, 128]  <a href="https://mobile.t

### Cleaning the Data Up

We will do some very simple cleanup to make dealing with our data easier. First, let's take out columns that we obviously won't need for our goals to make the data even more easy to analyze. Take a look at the initaldataframe.csv and determine what columns you think should be kept for our analysis. At this early stage it's usually a good idea to keep a column if in doubt. We can always trim it out later. Since we are keeping it simple we can hack and slash a bit more than usual. 


In [6]:
# Get a list of all column names
all_columns = df.columns.tolist()

# Print the list of column names
print(all_columns)


['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'entities.hashtags', 'entities.symbols', 'entities.user_mentions', 'entities.urls', 'metadata.iso_language_code', 'metadata.result_type', 'user.id', 'user.id_str', 'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 'user.entities.url.urls', 'user.entities.description.urls', 'user.protected', 'user.followers_count', 'user.friends_count', 'user.listed_count', 'user.created_at', 'user.favourites_count', 'user.utc_offset', 'user.time_zone', 'user.geo_enabled', 'user.verified', 'user.statuses_count', 'user.lang', 'user.contributors_enabled', 'user.is_translator', 'user.is_translation_enable

In [7]:
# Define the base column names you want to keep
# copy and paste from the output above to make sure you get the column names correct
base_columns_to_keep = ['created_at', 'id', 'full_text', 'in_reply_to_screen_name']

# Add the metadata and user columns to the list since those are somewhat interesting to us
columns_to_keep = base_columns_to_keep + [col for col in all_columns if col.startswith('metadata.') or col.startswith('user.')]

# Keep only the desired columns in the DataFrame
df = df[columns_to_keep]

# print out the results to a csv file to check the results
df.to_csv('limitedcolumns.csv', index=False)

In [8]:
# Looking at limitedcolumns.csv, it looks like we can safely remove some columns
# Let's remove the columns that have no data in them
df = df.dropna(axis=1, how='all')

# print out the results to a csv file to check the results
df.to_csv('limitedcolumns_2.csv', index=False)

In [9]:
# Now let's remove the columns that have the word "url" or "color" in them since they don't seem to be useful
df = df[df.columns.drop(list(df.filter(regex='url|color')))]

# print out the results to a csv file to check the results
df.to_csv('limitedcolumns_3.csv', index=False)

In [10]:
# drop the withheld_in_countries column because it doesn't have usable values
df.drop(columns=['user.withheld_in_countries'], inplace=True)

# let's also change all the columns with boolean values to be 1 or 0 instead of True or False
# this makes it easier to work with the data later
df = df.replace({True: 1, False: 0})

# print out the results to a csv file to check the results
df.to_csv('limitedcolumns_4.csv', index=False)

In [11]:
# we have trimmed the data down to 27 columns now and the data looks much more manageable
# a couple of more cleanup items and we will be ready to start analyzing the data
# let's convert all the column names to lowercase
df.columns = df.columns.str.lower()

# finally, let's replace any "." in the column names with "_" to be consistent
df.columns = df.columns.str.replace('.', '_')

# print out the results to a csv file to check the results
df.to_csv('limitedcolumns_5.csv', index=False)

### Sentiment and Emotion Analysis
Now on to the first set of analysis: 
For each Tweet, we need to find out the main sentiment (Positive, Neutral, Negative) and the main emotion (Joy, Surprise, Neutral, Sadness, Mistrust, and Disgust)
We will want to return, both, the predicted sentiment and emotion, as well as the score (ranging for -1 (negative) to 1 (positive) for Sentiment and 0 (disgust) to 1 (joy) for emotion).


In [ ]:
# define a function to take in our dataframe and return a new dataframe 
# with the columns for sentiment and emotion added to each tweet
def sentiment_emotion_analysis(df):
    # Initialize empty lists to hold sentiment and emotion data
    sentiments = []
    emotions = []

    # Loop through every tweet in the dataframe's 'full_text' column using Few-Shot Prompting
    for tweet in df['full_text']:
        # Define the prompt for GPT-3 to analyze sentiment and emotion
        # note we use the f-string (formatted string) to insert the tweet into the prompt
        # we should be doing few-shot chain-of-thought prompting but we don't have time
        myprompt=f"""
        Analyze the sentiment of the following tweet and provide a sentiment rating from -1 (completely negative) to 1 (completely positive) and all values in between. If the sentiment is neutral, provide a score of 0. Also, provide an emotion associated with the tweet, choosing from Joy, Surprise, Neutral, Sadness, Mistrust, and Disgust respectively. Rate the emotions from complete joy(1) down to complete disgust(0), assigning values to the emotions as follows: Joy = 1, Surprise = 0.80, Neutral = 0.60, Sadness = 0.40, Mistrust = 0.20, and Disgust = 0. 

        Here are some examples:

        Tweet: Who is making these decisions @SamsClub Do you hate your employees?? Are you kidding?! The @weatherchannel is here. Put it on tv, pay attention to what they are saying, and tell your employees to stay home! #HurricaneIdalia
        Sentiment: -0.8
        Emotion: Disgust

        Tweet: Sams club has frozen grill cheese sandwiches 🤔🤔sheer genius 😍😍
        Sentiment: 0.8
        Emotion: Joy

        Tweet: signed up for a sams club membership. this feel wayyy too grown for me
        Sentiment: 0.2
        Emotion: Sadness

        Tweet: I really need to go sams club
        sentiment: 0.0
        Emotion: Neutral

        Tweet: moms coming home@with sams club pizza!!
        Sentiment: 0.9
        Emotion: Surprise

        Provide the sentiment and emotion ratings in the following JSON format:
        {{"Sentiment": "<sentiment>", "Sentiment_Value": <sentiment_value>, "Emotion": "<emotion>", "Emotion_Value": <emotion_value>}}

        What is the sentiment and emotion of this tweet:
        "{tweet}"

        Answer: {{"Sentiment": "<sentiment>", "Sentiment_Value": <sentiment_value>, "Emotion": "<emotion>", "Emotion_Value": <emotion_value>}}
        
        """

        # Create an API call to OpenAI using the specified model, prompt, and parameters
        # With more time we can play with temperature and top_p to see if we can get better results
        response = openai.Completion.create(
            model="text-davinci-003",  # The model used for prediction
            prompt=myprompt,  # The prompt defined above
            temperature=0,  # Controls the randomness of the model's output
            max_tokens=100,  # The maximum number of tokens in the output
            top_p=0.50,  # The nucleus sampling parameter, which controls the randomness of token generation
            frequency_penalty=0.0,  # Penalizes the frequency of tokens in the output
            presence_penalty=0.0  # Penalizes the new tokens in the output
        )

        # Get the response text and remove the initial "Answer: "
        response_text = response['choices'][0]['text'].strip().replace("Answer: ", "")

        # Process the response text
        try:
            # Load the response into a JSON object
            response_json = json.loads(response_text)

            # Extract the sentiment and emotion data from the JSON object
            sentiment = response_json["Sentiment"]
            sentiment_value = response_json["Sentiment_Value"]
            emotion = response_json["Emotion"]
            emotion_value = response_json["Emotion_Value"]

            # Append the sentiment and emotion data to the corresponding lists
            sentiments.append((sentiment, sentiment_value))
            emotions.append((emotion, emotion_value))

        # If there's an error processing the response text, append "Error" and 0 to the data
        except ValueError as e:
            print(f"Error processing tweet: {tweet}")
            print(f"Response from API: {response_text}")
            sentiments.append(("Error", 0))
            emotions.append(("Error", 0))

    # Add the sentiment and emotion data to the dataframe as new columns
    df['sentiment'], df['sentiment_value'] = zip(*sentiments)
    df['emotion'], df['emotion_value'] = zip(*emotions)

    # Return the updated dataframe
    return df

# Call the function and update the dataframe
df = sentiment_emotion_analysis(df)
# Save the updated dataframe as a CSV file
df.to_csv('sentimentemotion.csv', index=False)
